In [38]:
!pip install pyspark py4j

In [39]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySparkColab").getOrCreate()

In [40]:
path = '/content/yellow_tripdata_2025-01.parquet'

df = spark.read.parquet(path,
                        header = True)
df.printSchema()
df.show(10)

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)

+--------+--------------------+---------------------+---------------+------

In [41]:
dfCleaned = df.dropna(subset = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'fare_amount'])
dfCleaned.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|          1.6|         1|                 N|         229|    

In [42]:
dfFiltered = dfCleaned.filter((dfCleaned.trip_distance > 0) & (dfCleaned.fare_amount > 0))
dfFiltered.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|          1.6|         1|                 N|         229|    

In [43]:
dfRenamed = dfFiltered.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')
dfRenamed = dfRenamed.withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')
dfRenamed = dfRenamed.withColumnRenamed('PULocationID', 'pickup_locID')
dfRenamed = dfRenamed.withColumnRenamed('DOLocationID', 'dropoff_locID')
dfRenamed.show(10)

+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+-------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|pickup_locID|dropoff_locID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+-------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1|2025-01-01 00:18:38|2025-01-01 00:26:59|              1|          1.6|         1|                 N|         229|          237

In [44]:
totalFair = dfRenamed.agg({'fare_amount': 'sum'})
totalPassenger = dfRenamed.agg({'passenger_count': 'sum'})
averageFair = totalFair.collect()[0][0] / totalPassenger.collect()[0][0]

print(f"Total fair: {totalFair.collect()[0][0]}")
print(f"Total passenger: {totalPassenger.collect()[0][0]}")
print(f"Average fair: {averageFair}")

Total fair: 59290060.92001146
Total passenger: 3680902
Average fair: 16.107481514045052


In [45]:
top

NameError: name 'top' is not defined